In [1]:
import pickle
import numpy as np
from tqdm import tqdm
from PIL import Image
import json

In [2]:
## 피클파일 없을 때 실행

# 메타데이터 및 텍스트 데이터 로드

file_name = 'data_256_cat5.pkl'
train_path = './data/%s'
metadata = pickle.load(open('./data/preprocessed.pkl', 'rb'))

target_size = 256

# clean_text에 시작, 끝 추가
# clean_text 기준 256글자 넘지 않도록 필터링
# 각 감정 클래스가 최대한 균등하도록 샘플 추출
# 학습 5000, 검증 1000, 테스트 1000

metadata['clean_text'] = metadata['clean_text'].apply(lambda x: '\t%s\n' %x.strip())
metadata = metadata[metadata['clean_text'].map(len) > 2]
metadata = metadata[metadata['hashtag'].map(len) > 2]
metadata = metadata[metadata['clean_text'].str.len() <= 256]
metadata = metadata.reset_index()

hashtag_data = []
for i, row in tqdm(metadata.iterrows()):
    _hash = row['hashtag'].replace("'", '"')
    pos = _hash.find('\\U')
    while pos != -1:
        _hash = _hash.replace(_hash[pos:pos+10], chr(int(_hash[pos+2:pos+10], 16)))
        pos = _hash.find('\\U')
    hashtag_data.append(json.loads(_hash))

metadata['hashtag'] = hashtag_data
del hashtag_data
metadata = metadata[metadata['hashtag'].map(len) <= 100]
    
sentiment_indices = [metadata[metadata['category1'] == cat].index for cat in metadata['category1'].unique()]
data_indices = []

print([len(x) for x in sentiment_indices])

start_idx = 28000
n_idx = 7000

i = 0
condition = True
while condition:
    for senti in sentiment_indices:
        if len(set(data_indices)) < (start_idx+n_idx):
            data_indices.append(senti[i])
        else:
            condition = False
            break
    i += 1

data_indices = data_indices[start_idx:]
metadata = metadata.iloc[sorted(list(set(data_indices)))]

def get_image_from_path(path):
    image = Image.open(train_path %path)
    size = image.size

    if size[0] <= size[1]:
        scaled_size = (target_size, int(size[1]/size[0]*target_size))
        crop_area = (0, int((scaled_size[1]-target_size)/2), target_size, int((scaled_size[1]+target_size)/2))
    else:
        scaled_size = (int(size[0]/size[1]*target_size), target_size)
        crop_area = (int((scaled_size[0]-target_size)/2), 0, int((scaled_size[0]+target_size)/2), target_size)

    image = image.resize(scaled_size)
    image = image.crop(crop_area)
    return image

def get_array_from_path(path):
    return np.asarray(get_image_from_path(path))

# 이미지 불러와서 데이터셋 생성
image_list = [get_array_from_path(path) for path in tqdm(metadata['image_path'])]
image_data = np.array(image_list)
del image_list
sentence_data = np.array(metadata['clean_text'])
hashtag_data = np.array(metadata['hashtag'])

355164it [00:32, 11073.25it/s]


[82167, 48382, 76784, 18710, 98109, 8838, 22162]


100%|██████████| 7000/7000 [03:28<00:00, 33.62it/s]


In [3]:
print(image_data.shape)
print(sentence_data.shape)
print(hashtag_data.shape)

(7000, 256, 256, 3)
(7000,)
(7000,)


In [4]:
from konlpy.tag import Komoran

komoran = Komoran()
use_tags = ['V', 'N', 'M', 'IC', 'XR']

original_tags = [komoran.pos(s) for s in tqdm(sentence_data)]
sentence_tags = []

for tags in tqdm(original_tags):
    sentence_tag = []
    for tag in tags:
        for use_tag in use_tags:
            if tag[1].startswith(use_tag):
                sentence_tag.append(tag)
    sentence_tags.append(sentence_tag)

100%|██████████| 7000/7000 [00:01<00:00, 5189.72it/s]


In [5]:
pickle.dump([['image_data', 'sentence_data', 'sentence_tags', 'hashtag_data', 'metadata'], [image_data, sentence_data, sentence_tags, hashtag_data, metadata]], open(file_name, 'wb'))